## 43. 1到n整数中1出现的次数

输入一个整数n，求1～n这n个整数的十进制表示中1出现的次数。例如，输入12，1～12这些整数中包含1的数字有1、10、11、12。但是1一共出现5次（算重复的1）。

[//]: # (<img src="images/img123.png" style="width: 400px;"/>)

### 找规律
1. 以41345为例子，先算出n一共有多少位（个、十、百、前、万...）--> 5位
2. **f(1)**: 0～9: 出现1次
3. **f(2)**: 10～99: 十位上1可以出现10次（10～19） + 个位上1可以分别出现1次，十位有0～9九个数：**f(1)**x9 = 19次   
4. **f(3)**: 100～999: 百位上1可以出现100次（100～199） + 剩下的十位和个位之前算过了，只要乘以前面的9个数字就行：（19+1)x9 = 280次  
5. **f(4)**: 1000～9999: 找到规律后可以直接算出：1000（千位）+ (**f(1)+f(2)+f(3)**)x9   
6. 所以general的公式是： $f(n) = 10^{n-1} + (f(1)+f(2)+ ... +f(n-1)) \cdot 9$
   - 最高位：0～39999: 出现了 f(1)+f(2)+f(3)+f(4)+ 10000 + (1+18+252+3168)x**3**
   - 4**0000**～4**1345**这里有点特殊，先算出4**0000**～4**0999**：f(1)+f(2)+f(3)，然后4**1000**～4**1345**千位上的“1”可以出现**345**次，所以总共是：f(1)+f(2)+f(3) + **345** 次
   - 41**000**～47**299**相当于0～299，出现了f(1)+f(2) + [100 + (18+1)x**1**] = 138次
   - 413**00**～473**39**相当于0～39，出现了f(1)+ [10 + 1x2] = 13次
   - 4134**0**～4734**5**相当于0～5，出现了1次

这个过程实际上就是，先求出最高位上0～**4**0000出现了多少次1，剩下求次高位0～**7**000，以此类推0～**3**00，0～**4**0，0～5。总共23095次

In [1]:
def NumberOf1Between1AndN_Solution(n):
    if not isinstance(n, int) or n < 0:
        raise Exception("Input should be strictly non-negative integer!")
    if n == 0:
        return 0

    count_ones = 0
    num_digits, digits = get_num_digits(n)

    while len(digits) > 0:
        count_ones += number_of_1_full_range(num_digits, digits)
        digits = digits[1:]
        num_digits -= 1

    return count_ones


def get_num_digits(n):
    """
    Find the how many digits does int n have and return each digits in a list.
    e.g. 21345 have 5 digits, return [2, 1, 3, 4, 5]
    """
    if not isinstance(n, int):
        raise Exception("Input of num_digits() should be strictly int!")
    count = 1
    digits = []

    while n // 10 > 0:
        count += 1
        # store digits reversely
        digits.insert(0, n % 10)
        # digits.append(n % 10)
        n = n // 10
    digits.insert(0, n % 10)
    # digits.append(n % 10)
    return count, digits

def number_of_1_full_range(num_digits, digits):
    """
    count how many numbers contains "1" from (e.g leading_digit=4):
    0~4(num_digits = 1)
    10~40 (num_digits = 2)
    100~400 (num_digits = 3)
    and so on
    """
    leading_digit = digits[0]
    if num_digits < 1 or not isinstance(num_digits, int):
        raise Exception("Input must be an int bigger than 0")
    if leading_digit < 0 or not isinstance(leading_digit, int) or leading_digit > 9:
        raise Exception("Input must be an int between 0~9")

    if num_digits == 1 and leading_digit != 0:
        return 1

    f = [1]
    # The formular is:
    # f(1)=1, f(2)=19
    # f(n) = 10^(n-1) + 9 * (f(1) + f(2) + ... + f(n-1))
    for i in range(2, num_digits):
        f_i = 10 ** (i-1) + 9 * sum(f)
        f.append(f_i)

    if leading_digit >= 2:
        result = 10 ** (num_digits - 1) + (leading_digit - 1) * sum(f) + sum(f)
    elif leading_digit == 1: # special case
        add_on = 0
        for i in range(1, len(digits[1:])+1):
            add_on += digits[i] * 10**(len(digits[1:]) - i)
        result = sum(f) + 1 + add_on
    else:
        result = 0

    return result

In [3]:

print("普通测试")
print(NumberOf1Between1AndN_Solution(5))
print(NumberOf1Between1AndN_Solution(10))
print(NumberOf1Between1AndN_Solution(55))
print(NumberOf1Between1AndN_Solution(99))

print("边界测试")
print(NumberOf1Between1AndN_Solution(0))
print(NumberOf1Between1AndN_Solution(1))

print("大数字测试")
print(NumberOf1Between1AndN_Solution(21345))
print(NumberOf1Between1AndN_Solution(47345))

print(NumberOf1Between1AndN_Solution(1000))
print(NumberOf1Between1AndN_Solution(999))

普通测试
1
2
16
20
边界测试
0
1
大数字测试
18821
29275
301
300
